In [37]:
from astropy import units as u
from astropy import constants as const
import numpy as np

# Angular Diameter

In [17]:
# observations:
theta_obs_H = 366 - 343 # observed angular diameter of H Beta
delta_x_obs_H = 362 - 349 # observed FWHM of H Beta
print(f"H: Theta = {theta_obs_H}, Delta_x = {delta_x_obs_H}")

theta_obs_He = 361.5 - 347.5 # observed angular diameter of He
delta_x_obs_He = 358 - 351 # observed FWHM of He
print(f"He: Theta = {theta_obs_He}, Delta_x = {delta_x_obs_He}")

theta_see = 359 - 349 # angular diameter of seeing
delta_x_see = 357 - 351 # FWHM of seeing
print(f"Seeing: Theta = {theta_see}, Delta_x = {delta_x_see}", end="\n\n")


# further calculations:
def get_delta_x_true(delta_x_obs):
    return np.sqrt(delta_x_obs**2 - delta_x_see**2)

def get_theta_true(delta_x_true):
    return (theta_see/delta_x_see)*delta_x_true

# H Beta:
delta_x_true_H = get_delta_x_true(delta_x_obs_H)
theta_true_H = (get_theta_true(delta_x_true_H)/1.82)*u.arcsec

delta_x_true_H = (delta_x_true_H/1.82)*u.arcsec # add unit
print(f"True Delta_x He = {delta_x_true_H}")
print(f"Angular distance H = {theta_true_H}", end="\n\n")

# He:
delta_x_true_He = get_delta_x_true(delta_x_obs_He)
theta_true_He = (get_theta_true(delta_x_true_He)/1.82)*u.arcsec

delta_x_true_He = (delta_x_true_He/1.82)*u.arcsec # add unit
print(f"True Delta_x He = {delta_x_true_He}")
print(f"Angular distance He = {theta_true_He}")

H: Theta = 23, Delta_x = 13
He: Theta = 14.0, Delta_x = 7
Seeing: Theta = 10, Delta_x = 6

True Delta_x He = 6.336572854214723 arcsec
Angular distance H = 10.56095475702454 arcsec

True Delta_x He = 1.9810721293758182 arcsec
Angular distance He = 3.3017868822930305 arcsec


# Absolute Fluxes

In [96]:
equiv_width_H = 94.4978*(u.erg/(u.cm**2*u.s))
equiv_width_He = 2.219203*(u.erg/(u.cm**2*u.s))

c = 0.09
abs_flux_H = 10**(-10.08+c)*(u.erg/(u.cm**2*u.s))
correction_abs_flux_He = 1.04*c * (u.erg/(u.cm**2*u.s))

rel_flux_H = equiv_width_H
rel_flux_He = equiv_width_He

f = theta_true_He**2/theta_true_H**2
alpha_He = abs_flux_H/rel_flux_H * f
abs_flux_He = abs_flux_H* rel_flux_He/rel_flux_H*f * 10**(correction_abs_flux_He.value)
# abs_flux_He = alpha_He * rel_flux_He * 10**(correction_abs_flux_He.value)


print(f"alpha He = {alpha_He}")
print(f"abs_flux_He = {abs_flux_He}")

alpha He = 1.0584491866079324e-13
abs_flux_He = 2.9138490141091155e-13 erg / (s cm2)


# Zanstra ratios

In [107]:
V_0 = 12.9 - 2.175*c
abs_flux_reference = 3.65e-9*10**(-0.4*V_0)*(u.erg/(u.cm**2*u.s*u.angstrom))
C = 5.197e-14
T_neb = 9600*u.K # nebula temperature

# Einstein coefficients:
A_H = 8.42e6/u.s
A_He = 142.4e6/u.s

# transmission frequencies:
nu_H = const.c/(4861.3*u.angstrom)
nu_He = const.c/(4686.0*u.angstrom)

# nuclear charges:
z_H = 1
z_He = 2

T_plus_H = T_neb/z_H**2
T_plus_He = T_neb/z_He**2
print(f"T_plus_H: {T_plus_H}")
print(f"T_plus_He: {T_plus_He}")

# from T_plus follows:
B_H = 3.58e-15*(u.cm**3*u.K**(3/2))
B_He = 1.44e-15*(u.cm**3*u.K**(3/2))

def alpha_2(z):
    x = 1.57890e5 * (z**2/(T_neb)).value
    return C*z*x**(1/2)*((1/2)*np.log(x) + 0.496*x**(-1/3) - 0.3412)*(u.cm**3/u.s)

def Zanstra(nu, B, A, z, abs_flux):
    return const.h.to(u.erg*u.s)*nu*((B*A)/(T_neb**(3/2)*alpha_2(z)*(abs_flux/abs_flux_reference)))


Zanstra_H = Zanstra(nu_H, B_H, A_H, z_H, abs_flux_H).decompose().to(u.erg/u.angstrom)
Zanstra_He = Zanstra(nu_He, B_He, A_He, z_He, abs_flux_He).decompose().to(u.erg/u.angstrom)
print(f"Zanstra_H = {Zanstra_H}")
print(f"Zanstra_He = {Zanstra_He}")

T_plus_H: 9600.0 K
T_plus_He: 2400.0 K
Zanstra_H = 1.4643535137899697e-16 erg / Angstrom
Zanstra_He = 6.067889503644172e-14 erg / Angstrom


In [172]:
T_eff = np.arange(40e3, 80e3, 10)*u.K
lambda_reference = 5480*u.angstrom
# lambda_ion_min_H = 911*u.angstrom
# lambda_ion_min_He = 227.8*u.angstrom
E_ion_H = 13.6*u.eV
E_ion_He = 4*13.6*u.eV

def get_Planck_wavelength(T):
    return (2*const.h*const.c**2/lambda_reference**5)/(np.exp((const.h*const.c/(lambda_reference*const.k_B*T))) - 1)

def get_N_ion(T, E_ion):
    nu_0 = ((E_ion)/(const.k_B*T))
    return (2/const.c**2 * ((const.k_B*T)/const.h)**3 * np.exp(-nu_0) * 
             (nu_0*(nu_0 + 2) + 2)).decompose()

def Zanstra_P_theo(T, E_ion):
    return (get_Planck_wavelength(T)/get_N_ion(T, E_ion)).to(u.erg/u.angstrom)

def solve_for_match_H(T):
    return np.abs(Zanstra_P_theo(T, E_ion_H)-Zanstra_H)

def solve_for_match_He(T):
    return np.abs(Zanstra_P_theo(T, E_ion_He)-Zanstra_He)

differences_H = [solve_for_match_H(temperature) for temperature in T_eff]
differences_He = [solve_for_match_He(temperature) for temperature in T_eff]

# print temperature with lowest difference
T_eff_H = T_eff[differences_H.index(min(differences_H))]
T_eff_He = T_eff[differences_He.index(min(differences_He))]
print(f"T_eff_H = {T_eff_H}") 
print(f"T_eff_He = {T_eff_He}") 

print(f"-log(Z_H) = {-np.log10(Zanstra_P_theo(T_eff_H, E_ion_H).value)}")
print(f"-log(Z_He) = {-np.log10(Zanstra_P_theo(T_eff_He, E_ion_He).value)}")



T_eff_H = 53000.0 K
T_eff_He = 55870.0 K
-log(Z_H) = 15.834279001023289
-log(Z_He) = 13.216887443050295


# Radius and Distance of the System

In [195]:
# M_sun = 1.989e30*u.kg
M_star = 0.534*const.M_sun
g = 10**(4.5)*(u.cm/u.s**2)

R_star = (np.sqrt(const.G*M_star/g)/(const.R_sun)).decompose()

F_mod_H = get_Planck_wavelength(53000*u.K)

d = np.sqrt(((R_star*const.R_sun)**2*F_mod_H)/abs_flux_reference).to(u.pc)

print(f"Radius star: {R_star}*R_sun")
print(f"Distance: {d}")

Radius star: 0.6804628850497475*R_sun
Distance: 1710.4975273361954 pc


# Diameter and Mass of the Nebula

In [213]:
m_H = 1.673e-27*u.kg
n_e = 5.1e3/u.cm**3
A_42 = A_H
B = B_H
n_p = n_e/1.13

# Radius of the nebula:
R_neb = 4.848e-6*(theta_true_H/2)*d.to(u.lyr)

# Mass of the nebula:
n_4 = n_e*n_p*B/T_neb**(3/2)
epsilon_times_V = ((4*np.pi*d**2*abs_flux_H)/(const.h*nu_H*n_4*A_42))

M_neb = (((1.4/1.13)*n_e*m_H*epsilon_times_V)/(const.M_sun)).decompose()

print(f"Radius: {R_neb}")
print(f"Mass: {M_neb}*M_sun")

Radius: 0.14281838690617266 arcsec lyr
Mass: 0.06318167979014339*M_sun
